In [1]:
# Imports

from argparse import Namespace
import pickle
from pathlib import Path
import numpy as np

from probo import NelderMeadAcqOptimizer, SimpleBo
from ackley import Ackley
from penn_sklearn import SklearnPenn

In [2]:
# Set seed

seed = 11
np.random.seed(seed)

In [3]:
# BO Setup

# define function
n_dim = 10
f = Ackley(n_dim)

# define model
model = SklearnPenn()

# define acqfunction
acqfunction = {'acq_str': 'ts', 'n_gen': 500}

# define acqoptimizer
domain = f.get_domain()
acqoptimizer = NelderMeadAcqOptimizer({'rand_every': 10, 'max_iter': 200, 'jitter': True}, domain)

# define initial dataset
n_init = 80
data = Namespace()
data.x = domain.unif_rand_sample(n_init)
data.y = [f(x) for x in data.x]

# define BO
n_iter = 3
bo = SimpleBo(f, model, acqfunction, acqoptimizer, data=data, params={'n_iter': n_iter}, seed=seed)

*[INFO] Model=SklearnPenn with params=Namespace(alpha=0.01, hls=(20, 30, 40), max_iter=500, n_ensemble=5, trans_x=False)
*[INFO] RealDomain with params=Namespace(dom_str='real', min_max=[[-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1]])
*[INFO] NelderMeadAcqOptimizer with params=Namespace(domain_params=Namespace(dom_str='real', min_max=[[-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1]]), init_str='bsf', jitter=True, jitter_val=0.1, k=2, max_iter=200, n_init_rs=0, n_rand=1, rand_every=10, rhobeg=0.5)
*[INFO] AcqFunction with params=Namespace(acq_str='ts', n_gen=500, trans_str='')
*[INFO] AcqOptDesigner with params=Namespace(n_rep=1, seed=11)
*[INFO] SimpleBo with params=Namespace(n_iter=3, print_x_str_len=30, reinit_designer=False, seed=11)
*[INFO] SimpleBo initialized with 80 observations


In [4]:
# Run BO

results = bo.run()

*[KEY] i: iteration, x: design, y: objective, min_y: minimum objective so far (* indicates a new min_y)
i: 0,    x: [-0.01532587707050702, -0.4092.. 	y: 14.5653,	min_y: 11.3729 
i: 1,    x: [-0.9762292667700637, -0.35882.. 	y: 11.2459,	min_y: 11.2459 *
i: 2,    x: [-1.0, -0.2843369365743596, -0.. 	y: 10.8346,	min_y: 10.8346 *
Minimum y = 10.834559311840747
Minimizer x = [-1.0, -0.2843369365743596, -0.358911111137036, 0.3857488300880684, 0.3308239776846102, -0.07510280362470115, -0.5673845611030341, -0.6101462744996327, -0.8874529462644731, -0.196990585675464]
Found at i = 2


In [5]:
# Save results

save_dir = Path(f'results/ackley{n_dim}_{seed}')
save_dir.mkdir(parents=True, exist_ok=True)

with open(save_dir / "results.pkl", "wb") as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)